# `compute_contact_times` Example

Testing the `compute_contact_times` function in `loc-gsopt/src/common/utils.py`

## Setup Imports

(not relevant to repo, can skip) 

Adding module path to run correctly in examples folder

In [1]:
import sys
import os
import numpy as np

# Add the path to the folder containing the module
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src


## Imports & Downloading latest Earth Orientation Data

In [2]:
# from common.sat_gen import make_tle
# from common.station_gen import gs_json
from common.utils import compute_contact_times, load_earth_data,mp_compute_contact_times
from common.sat_gen import satellites_from_constellation
from common.station_gen import return_bdm_gs


# Brahe Imports
import brahe as bh
import brahe.data_models as bdm
import brahe.access.access as ba
import multiprocessing as mp



%matplotlib inline
import matplotlib.pyplot as plt

# Setup info
load_earth_data('data/iau2000A_finals_ab.txt')

Loading the latest Earth Orientation Data


## Scenario Development

In [12]:
##### Trying to generate a satellite just through tles ######

# Create a TLE
epc_start = bh.Epoch(2025, 4, 1, 17, 23, 40.69) # This is the epoch of the orbital elements
epc_end = bh.Epoch(2025, 4, 8, 17, 23, 40.69)
satellites = satellites_from_constellation("CAPELLA")

ground_stations = []

# gs_list = [[37.09497170671422, 40.450882204488494], [61.63342713787408, 40.56768555325875], [55.32706006606998, 37.631986727162655], [-59.33476991738089, 40.378492817106824], [-67.29411618406957, 39.023695448552544]]
gs_list = [[22.69, 38.82], [143.45, 42.6], [-66.1,-33.2], [25.13, 36.99], [168.38,-46.53]]
gs_list = [[37.09448973202146, 40.449800713862366], [-12.871555503005073, -37.8840174526803], [130.91013047875538, 38.337363929137595], [78.30572166111048, -38.81669830641558], [85.97952156656379, 40.4554859605074]]
 #[[92.44410570135645, -90.0], [-73.01224079830891, -89.95426045890125], [-4.078723407808447, -89.95514937821778]]
for gs in gs_list:
    ground_stations.append(return_bdm_gs(gs[0], gs[1]))

plot = True
title="contact_times_chart.png"


In [13]:
epc_start.to_datetime()

datetime.datetime(2025, 4, 1, 17, 23, 40, 690000)

In [14]:
all_contacts, contact_times_seconds = mp_compute_contact_times(satellites,ground_stations,epc_start,epc_end, plot,title)


In [15]:
contact_times_seconds = [(contact.t_end - contact.t_start).total_seconds() for contact in all_contacts]


In [16]:
len(all_contacts)

1113

## Getting only one contact per satellite

In [17]:
from pyomo.environ import *
from itertools import combinations, groupby
import pyomo.kernel as pk

def contactExclusion(contacts):
    """
    Solves the contact exclusion problem using a pairwise overlap constraint approach.
    """

    model = ConcreteModel()

    # to call contacts in the correct order 
    contacts_order = {
        str(contact.id):i
        for i, contact in enumerate(contacts)
    }


    # Decision variables: x[c] = 1 if contact c is selected, 0 otherwise
    model.x = Var(range(len(contacts)), within=Binary)

    # Objective: Maximize total scheduled duration
    model.obj = Objective(
        expr=sum(model.x[i] * contact.t_duration 
                 for i, contact in enumerate(contacts)), 
        sense=maximize
    )

    # Constraint list to ensure no overlapping contacts
    model.constraints = ConstraintList()

    # Ensure that no two overlapping contacts for the same station are selected
    contacts_sorted_by_station = sorted(contacts, key=lambda cn: cn.station_id)

    for station_id, station_contacts in groupby(contacts_sorted_by_station, lambda cn: cn.station_id):

        # Convert groupby object to a sorted list (groupby creates an iterator)
        station_contacts = sorted(list(station_contacts), key=lambda cn: cn.t_start)

        # Test all pairs of contacts to check for overlap
        for x, y in combinations(station_contacts, 2):
            if x.t_start <= y.t_end and y.t_start <= x.t_end:
                model.constraints.add(model.x[contacts_order[str(x.id)]] + model.x[contacts_order[str(y.id)]] <= 1)

    # Ensure that no two overlapping contacts for the same satellite are selected
    contacts_sorted_by_satellite = sorted(contacts, key=lambda cn: cn.spacecraft_id)

    for sat_id, satellite_contacts in groupby(contacts_sorted_by_satellite, lambda cn: cn.spacecraft_id):

        # Convert groupby object to a sorted list
        satellite_contacts = sorted(list(satellite_contacts), key=lambda cn: cn.t_start)

        # Test all pairs of contacts to check for overlap
        for x, y in combinations(satellite_contacts, 2):
            if x.t_start <= y.t_end and y.t_start <= x.t_end:
                model.constraints.add(model.x[contacts_order[str(x.id)]] + model.x[contacts_order[str(y.id)]] <= 1)

    # miminum duration constraint
    for cn in contacts:
        if cn.t_duration <= 180:
            # Force all contacts with duration less than the minimum to be zero
            model.constraints.add(model.x[contacts_order[str(cn.id)]] == 0)

    # Solve the model
    solver = SolverFactory('cbc')
    solver.solve(model)

    # Extract selected contacts
    selected_contacts = [contact for i, contact in enumerate(contacts) if value(model.x[i]) > 0.75]
    contact_times_seconds = [(contact.t_end - contact.t_start).total_seconds() for contact in selected_contacts]

    return selected_contacts, contact_times_seconds


In [18]:
selected_contacts, contact_times_seconds = contactExclusion(all_contacts)

In [19]:
len(selected_contacts)

978

In [20]:
np.sum(contact_times_seconds)*1200000000


485073949231200.0

In [ ]:
np.sum(contact_times_seconds)-379118#*1200 - 


1059932282965200.0

In [22]:
1200000000*379118

454941600000000

In [151]:
454942360106400.0 - 747008165044799.9

-292065804938399.9

In [40]:
(403281039552000.0   - 435955897714800.0)/1200000000

-27229.048469

In [35]:

from geopy.distance import geodesic

geodesic((37.09497170671422, 40.450882204488494), (37.09497170671422, 60)).meters
# def penalty_gs_all(new_gs,current_gs_list, dist_penalty):
#     penalty_sum = 0
#     for current_gs in current_gs_list:
#         dist = geodesic((new_gs[1], new_gs[0]), (current_gs.geometry.coordinates[1], current_gs.geometry.coordinates[0])).meters
#         if dist < dist_penalty:
#             penalty_sum += dist_penalty - dist
#     return penalty_sum

1734845.1135049423

In [30]:
dist

0.0

In [36]:
403354179919200.0 - 403281039552000.0 

73140367200.0

In [37]:
73140367200.0/ 1200000000

60.950306